<a href="https://colab.research.google.com/github/rkrissada/google_ml_training/blob/master/2_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [0]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-394bd6ac587bed14'
PROJECT = 'qwiklabs-gcp-394bd6ac587bed14'
REGION = 'us-central1'

In [0]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [0]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [0]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

In [0]:
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,6392072535155213407,323758
1,8387817883864991792,331629
2,9183605629983195042,329975
3,328012383083104805,359891
4,8391424625589759186,364497


## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [0]:
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13559 examples in the train dataset and 3272 in the eval dataset


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [0]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.374463,True,25,1,38.0,774501970389208065
1,7.874912,True,24,1,39.0,774501970389208065
2,7.312733,False,36,1,39.0,774501970389208065
3,7.627994,False,19,1,40.0,774501970389208065
4,6.137669,True,24,1,36.0,774501970389208065


In [0]:
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13549.000000,13559.000000,13559.000000,13479.000000,1.355900e+04
mean,7.221074,27.299653,1.033631,38.580162,4.396483e+18
std,1.330929,6.127677,0.189070,2.626883,2.790354e+18
min,0.502654,12.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,12.187154,50.000000,4.000000,47.000000,9.183606e+18


In [0]:
import pandas as pd

def preprocess(df):
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [0]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.374463,True,25,Single(1),38.0,774501970389208065
1,7.874912,True,24,Single(1),39.0,774501970389208065
2,7.312733,False,36,Single(1),39.0,774501970389208065
3,7.627994,False,19,Single(1),40.0,774501970389208065
4,6.137669,True,24,Single(1),36.0,774501970389208065


In [0]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13554,9.250597,Unknown,24,Single(1),39.0,6637442812569910270
13555,8.560550,Unknown,26,Single(1),39.0,6637442812569910270
13556,8.703850,Unknown,30,Single(1),39.0,6637442812569910270
13557,6.499227,Unknown,16,Single(1),40.0,6637442812569910270
13558,8.937540,Unknown,25,Single(1),40.0,6637442812569910270


In [0]:
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26944.000000,26944.000000,26944.000000,2.694400e+04
mean,7.220448,27.298322,38.584694,4.395249e+18
std,1.331492,6.122371,2.610543,2.788041e+18
min,0.502654,12.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.108882e+18
max,12.187154,50.000000,47.000000,9.183606e+18


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [0]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [0]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6492 eval.csv
  26944 train.csv
  33436 total
==> eval.csv <==
7.4736706818,False,24,Single(1),42.0,1891060869255459203
7.25100379718,True,45,Single(1),39.0,7146494315947640619
6.56316153974,True,22,Single(1),39.0,6392072535155213407
6.0627122049999995,True,22,Single(1),34.0,1866590652208008467
8.2452885988,False,35,Single(1),39.0,8904940584331855459
7.1870697412,True,23,Single(1),39.0,6141045177192779423
8.50102482272,False,20,Single(1),41.0,8904940584331855459
7.1760466281,True,26,Single(1),43.0,8904940584331855459
5.43659938092,False,26,Single(1),37.0,7146494315947640619
6.4374980503999994,False,21,Single(1),38.0,74931465496927487

==> train.csv <==
7.3744626639,True,25,Single(1),38.0,774501970389208065
7.87491199864,True,24,Single(1),39.0,774501970389208065
7.31273323054,False,36,Single(1),39.0,774501970389208065
7.6279942652,False,19,Single(1),40.0,774501970389208065
6.13766937408,True,24,Single(1),36.0,774501970389208065
7.12534030784,False,21,Single(1),39.0,774501970389208065

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License